### Import Required Libraries and Set Up Environment Variables

In [ ]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import re
from urllib.parse import quote_plus


In [ ]:
# Set environment variables from the .env in the local environment    

def key_check(key_path=None):
    try:
        reply=load_dotenv(key_path,verbose=True,override=True)   
        assert reply , 'Dotenv is not found'
        nyt_api_key = os.getenv("NYT_API_KEY")
        tmdb_api_key = os.getenv("TMDB_API_KEY")
        assert nyt_api_key is not None, 'NYT_API_KEY not found in .env file'
        assert tmdb_api_key is not None, 'TMDB_API_KEY not found in .env file'
        responce=requests.get(f'https://api.nytimes.com/svc/mostpopular/v2/viewed/1.json?api-key={nyt_api_key}')
        assert responce.status_code == 200, f'The key provided failed to authenticate nyt_api_key {nyt_api_key} code {responce.status_code}'
        responce=requests.get(f'https://api.themoviedb.org/3/movie/11?api_key={tmdb_api_key}')
        assert responce.status_code == 200, f'The key provided failed to authenticate tmdb_api_key {tmdb_api_key} code {responce.status_code}'
    except Exception as e:
        # Handle potential errors in loading .env or missing API keys
        print(f'An error occurred: {e}')
        return(False)
    else:
        print('All keys loaded correctly')
        return (True)
my_env_path= 'C:\src\\ai\data-sourcing-challenge\.data-sourcing_challenge.env'
if key_check(my_env_path):
    nyt_api_key = os.getenv("NYT_API_KEY")
    tmdb_api_key = os.getenv("TMDB_API_KEY")
else:
    print ('fix Keys and rerun')

### Access the New York Times API

In [ ]:
# Sample from web page
# https://api.nytimes.com/svc/search/v2/articlesearch.json?q=new+york+times&page=2&sort=oldest&api-key=your-api-key 

#  Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = quote_plus('section_name:"Movies" AND type_of_material:"Review" AND headline:"love"')

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = quote_plus("headline,web_url,snippet,source,keywords,pub_date,byline,word_count")

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url=(f'{url}fq={filter_query}&begin_date={begin_date}&end_date={end_date}&fl={field_list}&sort={sort}&api-key={nyt_api_key}&page=')
display (query_url)

In [ ]:
# Create an empty list to store the reviews
reviews_list =[]
review_page=0
# loop through pages 0-19
start_page = 0
end_page = 20

for page in range(start_page,end_page):
    # Construct the query URL for the current page
    page_url = f'{query_url}{page}'
    # Attempt to make a "GET" request and parse the JSON response
    reviews = requests.get(page_url).json()
    try:
        # Check if the "docs" list is empty; if so, print a message and exit the loop
        if (reviews["response"]["docs"]):
            print(f'Checked page  {page}')
            # print ('sleep 12 seconds')
            time.sleep(12)                 
            # Add a twelve second pause between requests to adhere to API query limits
        else: 
            raise ValueError (f'No results on page {page} {review_page + 1} reviews found, stopping.')
    except ValueError as e:
        # Handle the case where no documents are found
        print(e)
        break
    except Exception as e:
        # Handle potential errors in the request or data processing
        print(f'An error occurred: {e}')
        break        
        # Otherwise, process each article in "docs"
    for review in reviews["response"]["docs"]:
        reviews_list.append(review)
        review_page += 1
        # print (f'review {review_page}' )
        
    

In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print (json.dumps(reviews_list[:5], indent=4))

In [ ]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
review_df=pd.json_normalize(reviews_list)
review_df

In [53]:
# Extract the title from the "headline.main" column and
# 
# Regular expression to match text enclosed by ‘ and ’
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019.
# import re

import re

# Define the lambda function
extract_text = lambda st: re.search(r"(?<=\u2018)[^‘’]*(?=\u2019)", st).group(0) if re.search(r"(?<=\u2018)[^‘’]*(?=\u2019)", st) else None

# Example usage
st = "This is a test ‘text to extract’ and some more text."
extracted_text = extract_text(st)

print(extracted_text)




# review_df['title'] = review_df['headline.main'].apply(
#    lambda st: re.search(r"(?:\u0020|^)\u2018(.+?)\u2019(?:\u003A|\u0020|$)", st))   
review_df['title'] = review_df['headline.main'].apply(
    lambda st: re.search(r"(?:\u0020|^)\u2018(.+?)\u2019(?:\u003A|\u0020|$)", st).group(1) if re.search(r"\u2018(.+?)\u2019", st) else None)
    
    # lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])

    #lambda st: st[st.find("(?:\u0020|^)\u2018")+1:st.find("\u2019(?:\u003A|\u0020|$")])
pd.describe_option('display.max_colwidth')
pd.set_option('display.max_colwidth', None)
display (review_df[['title','headline.main']])
pd.set_option('display.max_colwidth', 50)
pd.describe_option('display.max_colwidth')

text to extract
display.max_colwidth : int or None
    The maximum width in characters of a column in the repr of
    a pandas data structure. When the column overflows, a "..."
    placeholder is embedded in the output. A 'None' value means unlimited.
    [default: 50] [currently: 50]


,title,headline.main
0,The Attachment Diaries,"‘The Attachment Diaries’ Review: Love, Sick"
1,What’s Love Got to Do With It?,Review: ‘What’s Love Got to Do With It?’ Probably a Lot
2,You Can Live Forever,‘You Can Live Forever’ Review: Do You Love Me Now?
3,A Tourist’s Guide to Love,‘A Tourist’s Guide to Love’ Review: A Wearyingly Familiar Trip
4,Other People’s Children,‘Other People’s Children’ Review: True Romance
...,...,...
195,The Other Half,Review: A Combustible Pair Find Love in ‘The Other Half’
196,The Ottoman Lieutenant,"Review: Love as the World Wars, in ‘The Ottoman Lieutenant’"
197,Love & Taxes,Review: It’s All Mirth and Taxes in ‘Love & Taxes’
198,"Everybody Loves Somebody,","Review: ‘Everybody Loves Somebody,’ a Rom-Com With Bite"


display.max_colwidth : int or None
    The maximum width in characters of a column in the repr of
    a pandas data structure. When the column overflows, a "..."
    placeholder is embedded in the output. A 'None' value means unlimited.
    [default: 50] [currently: 50]


In [ ]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# this copy make it possible for me to rerun this without having to start from scratch
review_mod_df = review_df.copy(deep=True)

# Fix the "keywords" column by converting cells from a list to a string
review_mod_df['keywords'] = review_mod_df['keywords'].apply(extract_keywords)
# display(lmr_keyword_df[['title','keywords']].head(3))
#
#play time with style
styled_subset_df=review_mod_df.loc[:4,['title','keywords']]
styled_df = styled_subset_df.style.set_table_styles({
    'title': [{'selector': '',
                'props': [('width', '200px'), ('text-align', 'right')]}],
    'keywords': [{'selector': '',
                'props': [('width', '700px'), ('text-align', 'left')]}]
}, overwrite=False).hide(axis=0)
# display(styled_df)
display (review_mod_df)
display (styled_df)
del styled_df

In [ ]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles_list = review_mod_df['title'].to_list()
# print("Top 5 Titles:\n" + json.dumps(titles_list[:5], indent= 4, ensure_ascii=False)[1:-1])
display (titles_list)

### Access The Movie Database API

In [ ]:
# Prepare The Movie Database query
url_query   = "https://api.themoviedb.org/3/search/movie?query="
url_detail  = "https://api.themoviedb.org/3/movie/"

In [ ]:
# Create an empty list to store the results
tmbd_movies_list=[]

# Create a request counter to sleep the requests after a multiple of 50 requests
request_counter = 1
'''
https://developer.themoviedb.org/docs/rate-limiting
While our legacy rate limits have been disabled for some time, 
we do still have some upper limits to help mitigate needlessly high bulk scraping. 
They sit somewhere in the 50 requests per second range. 
This limit could change at any time so be respectful of the service we have built and respect the 429 if you receive one.
'''
# Loop through the titles
for title in titles_list:
    # Check if we need to sleep before making a request
    if (request_counter % 50) == 0:
        print (f'Sleeping at {request_counter} requests')
        time.sleep(1)
        request_counter += 1
    else:
    # Add 1 to the request counter
        request_counter += 1
    # Perform a "GET" request for The Movie Database
    response=requests.get(f'{url_query}{title}"&api_key={tmdb_api_key}')
    data = response.json()    
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found. 
    
    try:            
        if data['results'] != []:
            movie_id = data['results'][0]['id']
            # Make a request for the full movie details
            # Execute "GET" request with url
            detail_response = requests.get(f"{url_detail}{movie_id}?api_key={tmdb_api_key}")
            movie_detail = detail_response.json()

            # Extract the genre names into a list
            genres_list = [genre['name'] for genre in movie_detail['genres']]
            # Extract the spoken_languages' English name into a list
            spoken_languages = [spoken_language['english_name'] for spoken_language in movie_detail['spoken_languages']]
            # Extract the production_countries' name into a list
            production_countries = [production_countries['name'] for production_countries in movie_detail['production_countries']]        
            # Add the relevant data to a dictionary and
            # append it to the tmdb_movies_list list
            tmbd_movies_list.append({
                # 'movie_id'        : movie_id,
                'title'           : title,
                'original_title'  : (movie_detail['original_title']),
                'budget'          : (movie_detail['budget']),
                'original_language'  : (movie_detail['original_language']),
                'homepage'        : (movie_detail['homepage']),
                'overview'        : (movie_detail['overview']),
                'popularity'      : (movie_detail)['popularity'],
                'runtime'         : (movie_detail['runtime']),
                'revenue'         : (movie_detail['revenue']),
                'release_date'    : (movie_detail['release_date']),
                'vote_average'    : (movie_detail['vote_average']),
                'vote_count'      : (movie_detail['vote_count']),
                'gendre_list'     : genres_list,
                'spoken_languages': spoken_languages,
                'production_countries': production_countries})
                    
            # Print out the title that was found
            print (f'Found {title}')
        else: 
            raise ValueError (f"{title} not found.")   
    except ValueError as e:
        # Handle the case where no documents are found
        print(e)
    except requests.exceptions.RequestException as e:
    # This catches all exceptions that are requests-related
    # Including connection errors, timeouts, etc.
        print("A network error occurred. Please try again later.")
        print(e)



In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print (json.dumps(tmbd_movies_list[:5],indent=4))

In [ ]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmbd_movies_list)
tmdb_df

### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(tmdb_df,review_mod_df, on= 'title', how='inner')
merged_df

In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing

columns_to_fix = [col for col in merged_df.columns if merged_df[col].map(lambda x: isinstance(x, list)).any()]
print (columns_to_fix)
# Create a list of characters to remove
def clean_list_string(s):
    if isinstance(s, list):
        # Convert list to string and remove unwanted characters
        return str(s).strip('[]').replace('"', '').replace("'", "")
    return s

# Apply the cleaning directly, no need for preliminary list check
for col in columns_to_fix:
    merged_df[col] = merged_df[col].apply(clean_list_string)

# Display the fixed DataFrame
print (merged_df.head())


In [ ]:
# Drop "byline.person" column

merged_df.drop(columns=['byline.person'], inplace=True)

In [ ]:
# Delete duplicate rows and reset index
duplicates = merged_df[merged_df.duplicated('title')]
display (duplicates)
merged_df = merged_df.drop_duplicates(keep='first').reset_index(drop=True)
merged_df

In [ ]:
# Export data to CSV without the index
merged_df.to_csv('tmdb_reviews.csv',index=False, encoding='utf-8')